In [1]:
# Libraries

import geemap, ee # Import geemap and earth engine together

print("Libraries imported")

# Authenticate Google Earth Engine and initialize project

ee.Authenticate() # Authenticate Google Earth Engine account
ee.Initialize(project="geog-581-483717") # Initialize GEOG 581 project

print("Authenticated and initialized")

/Users/marcel/Desktop/geog581-gee-notebook/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.4) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Libraries imported
Authenticated and initialized


In [2]:
band_names = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'])

image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318').select(band_names)

region = image.geometry()
Map = geemap.Map(ee_initialize=False)
Map.centerObject(region, 10)
Map.addLayer(ee.Image().paint(region, 0, 2), {}, 'Region')
Map.addLayer(image, { "bands": ['B5', 'B4', 'B2'], min: 0, max: 20000 }, 'Original Image')
Map

Map(center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['position', 'transparent…

In [3]:
scale = 30

mean_dict = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=scale,
    maxPixels=1e9,
)

means = mean_dict.toImage(band_names)
centered = image.subtract(means)

In [4]:
def prepend_band_names(prefix, band_list):
    band_list = ee.List(band_list)
    return band_list.map(lambda name: ee.String(prefix).cat(ee.String(name)))

new_bands = prepend_band_names("PC_", band_names)
print(new_bands.getInfo())

['PC_B2', 'PC_B3', 'PC_B4', 'PC_B5', 'PC_B6', 'PC_B7', 'PC_B10', 'PC_B11']


In [8]:
def get_principal_components(centered, scale, region):
    arrays = centered.toArray()

    covar = arrays.reduceRegion(
        reducer=ee.Reducer.centeredCovariance(),
        geometry=region,
        scale=scale,
        maxPixels=1e9
    )
    
    covar_array = ee.Array(covar.get('array'))
    eigens = covar_array.eigen()
    eigen_values = eigens.slice(1, 0, 1)
    eigen_vectors = eigens.slice(1, 1)
    array_image = arrays.toArray(1)
    principal_components = ee.Image(eigen_vectors).matrixMultiply(array_image)
    sd_image = ee.Image(eigen_values.sqrt()).arrayProject([0]).arrayFlatten([prepend_band_names('sd', band_names)])

    return principal_components.arrayProject([0]).arrayFlatten([prepend_band_names('sd', band_names)]).divide(sd_image)

pc_image = get_principal_components(centered, scale, region)

In [9]:
pc_band_names = pc_image.bandNames().getInfo()

for i, name in enumerate(pc_band_names):
    Map.addLayer(pc_image.select(i), {"min": -2, "max": 2}, name)

Map

Map(bottom=101903.0, center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['positi…